In [ ]:
from robot import *
from tqdm import tqdm
import pyrosim
import numpy as np

np.random.seed(9)

G = 500
MRATE = 0.01
POP_SIZE = 50
EVAL_TIME = 1000

def init():
    return np.random.randn(POP_SIZE, 5, 8)

def mutate(population):
    return np.random.randn(POP_SIZE, 5, 8) * MRATE + population

def evaluate(population):
    sims = []
    sensors = []
    fits = []
    for i in range(0, POP_SIZE, 5):
        for j in range(i, i+5):
            sims.append(
                pyrosim.Simulator(eval_time=EVAL_TIME, play_blind=True)
            )
            sensors.append(
                make_robot(sims[j], population[j])
            )
            run(sims[j])

        for j in range(i, i+5):
            fits.append(
                fitness(sims[j], sensors[j])
            )
    return fits

def survival_of_fitest(parents, p_fits, children, c_fits):
    fitest = np.zeros((POP_SIZE, 5, 8))
    for i in range(POP_SIZE):
        if p_fits[i] >= c_fits[i]:
            fitest[i] = parents[i]
        else:
            fitest[i] = children[i]
    return fitest
    


parents = init()
p_fits = evaluate(parents)

for g in tqdm(range(G)):
    children = mutate(parents)
    c_fits = evaluate(children)
    
    parents = survival_of_fitest(parents, p_fits, children, c_fits)
    p_fits = evaluate(parents) 

In [ ]:
idx = np.argmax(p_fits)
best_indv = parents[idx]

sim = pyrosim.Simulator(eval_time=EVAL_TIME, play_blind=False, play_paused=True)
sensor = make_robot(sim, best_indv)
run(sim)
print(fitness(sim, sensor))